<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/experiment/%E7%8F%BE%E6%9C%89%E6%A8%A1%E7%B5%84%E6%BA%AB%E5%BA%A6%E5%8F%8A%E8%8B%AF%E7%9A%84%E9%97%9C%E4%BF%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/skywalker0803r/c620.git
!pip install autorch > log.txt
!pip install optuna > log.txt

Cloning into 'c620'...
remote: Enumerating objects: 2266, done.
remote: Counting objects: 100% (1084/1084), done.
remote: Compressing objects: 100% (626/626), done.
remote: Total 2266 (delta 706), reused 688 (delta 449), pack-reused 1182
Receiving objects: 100% (2266/2266), 59.93 MiB | 20.01 MiB/s, done.
Resolving deltas: 100% (1484/1484), done.


# import

In [2]:
from c620.FV2 import AllSystem
from c620.configV2 import config
import joblib
import numpy as np
import pandas as pd
import os
os.chdir('/content/c620')
print(os.getcwd())
!git pull

/content/c620
Already up to date.


# demo data

In [3]:
demo = joblib.load('./data/demo.pkl')
print(demo.keys())
icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']
icg_input.head()

dict_keys(['icg_input', 'c620_feed', 't651_feed', 'c620_op', 'c620_wt', 'c660_op', 'c660_wt', 'c670_op', 'c670_wt'])


,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
137-026,188.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,77.0,0.0911,20.854397,74.689774,70.0,920.0,0.01,10.000004,36.0


# 這一筆樣本的NA in Benzene_ppmw只有920 期望能拉到 980

In [4]:
icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
137-026,920.0


In [5]:
icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
137-026,70.0


In [6]:
icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw
137-026,10.000004


# 1.試算模式

In [7]:
f = joblib.load('/content/c620/model/allsystem.pkl')

In [8]:
demo['icg_input']

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
137-026,188.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,77.0,0.0911,20.854397,74.689774,70.0,920.0,0.01,10.000004,36.0


In [14]:
table = pd.DataFrame()
for i in [70,80,90]:
  icg_input = demo['icg_input']
  icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'] = i 
  c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = f.inference(demo['icg_input'],demo['c620_feed'],demo['t651_feed'])
  table = table.append(c620_op.iloc[:,-2:].join(icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']))
table

,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
137-026,177.997787,192.847977,70
137-026,177.220886,192.419495,80
137-026,175.391632,192.097565,90


# 推薦模式

In [16]:
f = joblib.load('/content/c620/model/allsystem.pkl')

In [20]:
table = pd.DataFrame()
for i in [70,80,90]:
  icg_input = demo['icg_input']
  icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'] = i 
  c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op,_,_,_ = f.recommend(demo['icg_input'],demo['c620_feed'],demo['t651_feed'],search_iteration=10)
  table = table.append(c620_op.iloc[:,-2:].join(icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']))
table

[I 2021-04-29 10:00:09,044] A new study created in memory with name: no-name-6ac3dee1-bfee-4ee4-8698-abb3b49c778c
[I 2021-04-29 10:00:09,134] Trial 0 finished with value: 1.0528475187249213 and parameters: {'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 176.74050008780713, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 194.0174840422207, 'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr': 8.403867799249873}. Best is trial 0 with value: 1.0528475187249213.
[I 2021-04-29 10:00:09,211] Trial 1 finished with value: 1.04385870610351 and parameters: {'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 177.82087281233308, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 193.3585811337119, 'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr': 0.9759448074513549}. Best is trial 1 with value: 1.04385870610351.
[I 

bz_error: 0.16187149864398975
nainbz_error: 5.142353345573497
tol_error: 1.722620902152908


[I 2021-04-29 10:00:11,343] A new study created in memory with name: no-name-c0391506-9881-47b9-b6b1-130e721b35c1
[I 2021-04-29 10:00:11,432] Trial 0 finished with value: 1.1036645474689684 and parameters: {'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 175.5117472482969, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 193.13963559336736, 'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr': 3.187461590947274}. Best is trial 0 with value: 1.1036645474689684.
[I 2021-04-29 10:00:11,514] Trial 1 finished with value: 1.1102315373664142 and parameters: {'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 177.97683834185773, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 192.99343904684204, 'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr': 0.23569525178057713}. Best is trial 0 with value: 1.103664547468968

bz_error: 0.08719654351109796
nainbz_error: 6.6067787303790055
tol_error: 2.1672126135122207


[I 2021-04-29 10:00:13,572] A new study created in memory with name: no-name-ce2b7982-bfc2-4881-82cb-fffe8fe5b8da
[I 2021-04-29 10:00:13,663] Trial 0 finished with value: 1.1406558022732947 and parameters: {'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 177.5437347620732, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 193.08423080865484, 'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr': 4.332758381395966}. Best is trial 0 with value: 1.1406558022732947.
[I 2021-04-29 10:00:13,750] Trial 1 finished with value: 1.0187554234150624 and parameters: {'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 176.00466647176327, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 192.29126509971795, 'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr': 0.05068338346458158}. Best is trial 1 with value: 1.018755423415062

bz_error: 0.07918735430286006
nainbz_error: 9.477842738494928
tol_error: 0.012903134342430178


,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
137-026,177.566410,192.774577,70
137-026,178.094662,192.510274,80
137-026,174.878366,192.082037,90
